In [43]:
import requests
import os
import json
from dotenv import load_dotenv
import pathlib
import shutil

In [44]:
load_dotenv()

True

In [45]:
addr2label = {
    "1-tyome": ["1-chōme", "1 Chome"],
    "2-tyome": ["2-chōme", "2 Chome"],
    "3-tyome": ["3-chōme", "3 Chome"],
    "4-tyome": ["4-chōme", "4 Chome"],
    "5-tyome": ["5-chōme", "5 Chome"],
    "6-tyome": ["6-chōme", "6 Chome"],
    "7-tyome": ["7-chōme", "7 Chome"],
}

In [46]:
def move_file_in_label_folder(label: str, latitude: float, longitude: float):
    name = f"{latitude:.8f}-{longitude:.8f}"
    coord_path = pathlib.Path("data") / "coords" / name / f"{name}.png"
    if not coord_path.exists():
        print(f"{coord_path.as_posix()} does not exist")
        return
    dest_path = pathlib.Path(f"{coord_path.parent.parent.parent.as_posix()}/addrs/{label}")
    dest_path.mkdir(parents=True, exist_ok=True)
    dest_path = dest_path / f"{name}.png"
    shutil.copy(coord_path, dest_path)

In [47]:
def call(latitude: float, longitude: float):
    key = os.environ["GOOGLE_MAPS_GEOCODE_API_KEY"]
    req = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={key}"
    res = requests.get(req)
    content = res.content.decode()
    content = json.loads(content)
    results = content["results"]
    result = results[0]
    formatted_address: str = result["formatted_address"]
    print("addr", formatted_address)
    label = None
    for l, addrs in addr2label.items():
        if label is not None:
            break
        for addr in addrs:
            if addr in formatted_address:
                label = l
                break
    if label is None:
        err_message = f"can't find label of {formatted_address}. latitude: {latitude:.8f}. longitude: {longitude:.8f}"
        print(err_message)
        with open("failed_results.txt", mode="a+") as f:
            f.write(err_message)
        return
    print("label", label)
    move_file_in_label_folder(label, latitude, longitude)

In [48]:
root = pathlib.Path("data/coords")
files = list(root.iterdir())
print("len", len(files))
files.reverse()
for p in files:
    lat, lng = map(float, p.name.split("-"))
    print(lat, lng)
    call(lat, lng)

len 1512
35.539103 139.64983014
addr Japan, 〒223-0052 Kanagawa, Yokohama, Kohoku Ward, Tsunashimahigashi, 5-chōme−20−１０ キャッスル２１
label 5-tyome
35.55130085 139.64219729
addr 1-chōme-45-1 Hiyoshihonchō, Kohoku Ward, Yokohama, Kanagawa 223-0062, Japan
label 1-tyome
35.54164422 139.64168843
addr 4-chōme-3-10 Tsunashimahigashi, Kohoku Ward, Yokohama, Kanagawa 223-0052, Japan
label 4-tyome
35.55689154 139.65695414
addr Japan, 〒211-0025 Kanagawa, Kawasaki, Nakahara Ward, Kizuki, 4-chōme−51−２８ すぎな荘
label 4-tyome
35.54062773 139.643215
addr 4-chōme-7-27 Tsunashimahigashi, Kohoku Ward, Yokohama, Kanagawa 223-0052, Japan
label 4-tyome
35.54825139 139.657463
addr 4-chōme-6-15 Minamikase, Saiwai Ward, Kawasaki, Kanagawa 212-0055, Japan
label 4-tyome
35.54062773 139.64575929
addr 4 Chome-10 Tsunashimahigashi, Kohoku Ward, Yokohama, Kanagawa 223-0052, Japan
label 4-tyome
35.55638329 139.64728586
addr Japan, 〒223-0061 Kanagawa, Yokohama, Kohoku Ward, Hiyoshi, 2-chōme−7−１６ グレイトハウス日吉
label 2-tyome
35.559